# Libraries

In [150]:
import pandas as pd
import numpy as np
import nltk
from nltk.util import ngrams
import pymorphy2
import ssl
import re

In [86]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt')
SENT_DETECTOR = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to /Users/yermakhan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [87]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

# Functions

In [90]:
def return_bigrams(text):
    double_word = []
    for grams in ngrams(text.split(), 2):
        double_word.append(grams)
    return double_word
   # double_word = double_word[:-1]
    
def return_trigrams(text):
    triple_word = []
    for grams in ngrams(text.split(), 3):
        triple_word.append(grams)
    return triple_word
   # double_word = double_word[:-1]

def return_fograms(text):
    quadro_word = []
    for grams in ngrams(text.split(), 4):
        quadro_word.append(grams)
    return quadro_word

In [89]:
def counter(df1):
    all_bigrams = []
    for i in df1.index: # проходимся и добавляем по каждому слову в листе в каждой строке
        for j in df1['bigram_list'][i]:
            all_bigrams.append(j)
    
    all_trigrams = []
    for i in df1.index:
        for j in df1['trigram_list'][i]:
            all_trigrams.append(j)

    all_fograms = []
    for i in df1.index:
        for j in df1['fogram_list'][i]:
            all_fograms.append(j)

    temp_df_bi = pd.DataFrame(all_bigrams).value_counts().head(60).to_frame('count').reset_index()
    temp_df_tri = pd.DataFrame(all_trigrams).value_counts().head(30).to_frame('count').reset_index()
    temp_df_four = pd.DataFrame(all_fograms).value_counts().head(30).to_frame('count').reset_index()

    return temp_df_bi, temp_df_tri, temp_df_four

In [332]:
def company_in_text(text):
    prob_thresh = 0.4
    morph = pymorphy2.MorphAnalyzer()
    company_name = ''
    for word in word_tokenize(text):
        for p in morph.parse(word):
            if 'Name' in p.tag and p.score >= prob_thresh:
                if occurences(text, company_dict):
                    company_name = text.split('компания ', maxsplit=1)[-1].split(' ', 2)[:2]
                    company_name = " ".join(company_name)
                    return company_name
    return company_name

In [311]:
def name_in_text(text):
    prob_thresh = 0.4
    morph = pymorphy2.MorphAnalyzer()
    name = ''
    for word in word_tokenize(text):
        for p in morph.parse(word):
            if 'Name' in p.tag and p.score >= prob_thresh:
                name = word
                return name.title()
    return name

In [145]:
def occurences(text, dictionary):
    in_dict = any([word in text for word in dictionary])
    if in_dict == True:
        return True
    else:
        return False

In [93]:
def listToString(s):
    str1 = ""
    for ele in s:
        str1 += ele + ' '
    return str1

In [331]:
greetings = {'здравств', 'добрый'}
introduction = {'звать'}
company_dict = {'компания'}
goodbye = {'хороший вечер', 'до свидание', 'весь добрый'}


# Proccessing

In [388]:
df = pd.read_csv('test_data.csv')

df['is_greeting'] = ''
df['is_introduction'] = ''
df['manager_name'] = ''
df['company_name'] = ''
df['is_goodbye'] = ''

df['lemmatized'] = ''


#tokenizing and lemmatizing text
for i in df.index:
    doc2 = Doc(df['text'][i])
    doc2.segment(segmenter)
    doc2.tag_morph(morph_tagger)
    lemmatized_text = []
    for token in doc2.tokens:
        token.lemmatize(morph_vocab)
        lemmatized_text.append(token.lemma)
    df['lemmatized'][i] = lemmatized_text

# parsing name of 'manager' and 'company' from each line of dataset
for i in df.index:
    if df['role'][i] == 'client' and df['line_n'][i] < 4:
        df['manager_name'][i] = name_in_text(df['text'][i])
        df['company_name'][i] = company_in_text(df['text'][i])

df['lemmatized_string'] = df.apply(lambda x: listToString(x['lemmatized']), axis = 1)
df = df[df['lemmatized_string'].isnull() == False]
df['bigram_list'] = df.apply(lambda x: return_bigrams(x['lemmatized_string']), axis = 1)
df['trigram_list'] = df.apply(lambda x: return_trigrams(x['lemmatized_string']), axis = 1)
df['fogram_list'] = df.apply(lambda x: return_fograms(x['lemmatized_string']), axis = 1)

df['is_greeting'] = df['lemmatized_string'].apply(lambda x: True if occurences(x, greetings) else False)
df['is_introduction'] = df['lemmatized_string'].apply(lambda x: True if occurences(x, introduction) else False)
df['is_goodbye'] = df['lemmatized_string'].apply(lambda x: True if occurences(x, goodbye) else False)



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

# Saving file

In [389]:
df_prod = df.drop(df.columns[[9,10,11,12,13]], axis = 1, inplace = False)
df_prod.to_csv('data.csv')